# Installing roboflow and Downloading Dataset

In [ ]:
%pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="BN5iZ97tNEfphHurQeEN")
project = rf.workspace("pedro-azevedo-3c9ol").project("bdd100k-3zgda")
version = project.version(5)
dataset = version.download("yolov11")

## Changing yaml class names of pedestrian to person for  maintaining consistent class names as yolo pretrained

In [ ]:
content = """train: ../train/images
val: ../valid/images
test: ../test/images

nc: 4
names: ['car', 'person', 'traffic light', 'traffic sign']

roboflow:
  workspace: pedro-azevedo-3c9ol
  project: bdd100k-3zgda
  version: 5
  license: CC BY 4.0
  url: https://universe.roboflow.com/pedro-azevedo-3c9ol/bdd100k-3zgda/dataset/5"""

# Write to the file
with open('/kaggle/working/BDD100k-5/data.yaml', 'w') as file:
    file.write(content)

# Verify the changes
print("Updated data.yaml contents:")
with open('/kaggle/working/BDD100k-5/data.yaml', 'r') as file:
    print(file.read())

In [ ]:
# Install required packages
%pip install ultralytics
%pip install -U ipywidgets
%pip install albumentations  # for additional augmentations

# Import required packages
from ultralytics import YOLO
import torch

# Load YOLO model
model = YOLO("yolo11l.pt")


# Training configuration
results = model.train(
    data="/kaggle/working/BDD100k-5/data.yaml",
    epochs=30,              # Increased epochs since metrics were still improving in previous runs
    imgsz=1280,            # Increased image size for better small object detection
    batch=8,               # Reduced batch size due to larger images
    device=[0,1],          # Multi-GPU training
    
    # Optimization
    optimizer="AdamW",     # Changed to AdamW for better convergence
    lr0=0.01,             # Initial learning rate
    lrf=0.0001,            # Final learning rate - model will decrease from lr0 to lrf
    weight_decay=0.0005,   # Increased weight decay to reduce overfitting, range is 0.0001 to 0.001, Acts as a regularization technique

    
    # Data handling
    cache=True,             # Cache images in RAM
    augment=True,           # Enable default augmentations

    mixup=0.1,              # Add mixup augmentation, 10% of images will use mixup,Takes two images and their labels
                            # Blends them together with a random ratio
                            # Creates new training samples
                            # Helps prevent memorization and overfitting


    mosaic=1.0,             # Enable mosaic augmentation
                            # 100% of images will use mosaic
                            # Combines 4 different images into one
                            # Randomly scales and places them in a 2x2 grid
                            # Great for small object detection
                            # Increases context for the model
    
    # Early stopping and model saving
    patience=7,            # Increased patience for early stopping
    save_period=5,         # Save checkpoints every 5 epochs
    freeze=12,            # Freeze first 12 layers initially
    
    # Class handling
    overlap_mask=True,      # Better handling of overlapping objects
    
    box=7.5,                # Box loss gain
                            # Range: typically 5.0 to 10.0


    cls=0.5,                # Class loss gain
                            # Range: typically 0.3 to 1.0

    dfl=1.5,                # DFL loss gain
                            # Distribution Focal Loss - helps with precise boundary regression
                            # DFL loss deals with discerning between objects that are closely alike or challenging to differentiate,
                            # by better understanding their features and spatial info.
                            # dealing with class imbalance by nature of focusing on important samples

                            # Range: typically 1.0 to 2.0

                            # Higher values (e.g., 1.8):
                                # More precise boundary predictions
                                # Better for tasks requiring exact boundaries


                            # Lower values (e.g., 1.2):
                                # Less strict on boundary precision
                                # Use when approximate boundaries are acceptable
    
    # Additional parameters
    close_mosaic=10,      # Disable mosaic augmentation for final 10 epochs

    label_smoothing=0.1,  # Add label smoothing
                            # Range: typically 0.0 to 0.3
                            # Prevents Overconfidence:
                            # Without smoothing: Model might be 100% sure it's a car
                            # With smoothing: Model might be 90% sure it's a car
                            # Makes model more robust to errors in training labels

                            # Example
                            # Without Label Smoothing (traditional one-hot):
                            # car = 1.0
                            # truck = 0.0
                            # person = 0.0

                            # # With label_smoothing=0.1:
                            # car = 0.9     # (1 - 0.1)
                            # truck = 0.05  # (0.1 / number_of_classes)
                            # person = 0.05



    cos_lr=True,            # Use cosine learning rate scheduling

                            # Learning rate follows a cosine curve: True
                            #  High   ─▲
                            #         │    ╭─────╮
                            #  Rate   │ ╭──╯     ╰──╮
                            #         │╭╯          ╰╮
                            #  Low    ▼╯            ╰─
                            #          Start → End of Training , Better for longer training (>=30 epochs)

                            # Learning rate decreases linearly: False
                            #  High   ─▲
                            #         │╲
                            #  Rate   │ ╲
                            #         │  ╲
                            #  Low    ▼───╲─
                            #          Start → End of Training , Shorter training runs (<30 epochs)
)

# Export the model in multiple formats
model.export(format='onnx', dynamic=True, simplify=True)  # ONNX format
model.export(format='engine', dynamic=True)  # TensorRT format for faster inference

In [ ]:
%zip -r /kaggle/working/runs/detect/train.zip /kaggle/working/runs/detect/train